In [2]:
import random as random
import numpy as np

#Abstract

A lot of phenomena in nature exhibit linear dependance, therefore require using linear algebra. There are several techniques that increase the efficiency of computations, like LU and Cholesky decompositions. The purpose of this study is to find LU decomposition of matrix $A$, use it to calculate inverse and to compare this result to inverse function in numpy library. It was found that the condition number of matrix A is $10^{3}$ which is consistent with error of $10^{-13}$ we get from calculating matrix inverse. Also, nunpy implementation was 40 times faster than my code.

#Introduction

One of the standard ways to solve a system of linear equations $Ax=b$ is Gauss elimination. Oftetimes we need to solve this system for several different $b$ vectors and in this case Gauss elimination is not very efficient. If we are able to decompose $A$ into a product of lower-triangular matrix $L$ and upper-triangular matrix $U$, we gain a boost in computational efficiency.  

It is possible to compute inverse of matrix $A$ using LU decompositions. If $b^i$ is a $i$-th column of an identity matrix, than the solution to $Ax^i=b^i$ is the $i$-th column of $A^{-1}$

Condition number relates how an error in initial data affects the final result, for matrices it is $cond=||A||*||A^{-1}||$ where $||A||$ is a norm of matrix, defined as $||A||=\sqrt{\Sigma_{ij}a_{ij}^2}$
If condition number is much higher than unity, than the problem is ill-conditioned and results to a huge error.


#Methods

In part (1) we need a function generate_a() that generates 50-by-50 matrix $A$ with random entries in range $(0,1)$ using a pre-determined seed.

In [3]:
def generate_a():
  random.seed(1007092020)
  A=np.zeros((50,50))
  for i in range(50):
    for j in range(50):
      A[i,j]=random.random()
  return A

In part (2) we need several things. First function doolitle(a) employs LU decomposition using Doolitle method.

We split $LUx=b$ in two parts:
$Ly=b$ and $Ux=y$. Function doolforward(L,b) solves the forward substition and returns $y$. Then function doolbackward(U,y) performs backward substitution and returns $x$

In order to find inverse we use function inverse(A) which uses previously mentioned functions.

In [4]:
def doolittle(a):
  ac=np.copy(a)
  n=len(np.diag(a))
  l=np.zeros(a.shape)
  u=np.zeros(a.shape)
  for k in range(n):
    l[k,k]=1
  for q in range(n):
    u[0,q]=ac[0,q]
  for i in range(0,n-1):
    for j in range(i+1,n):
      m=ac[j,i]/ac[i,i]
      l[j,i]=m
      ac[j,:]=ac[j,:]-m*ac[i,:]
      u[j,:]=ac[j,:]
  return l,u

def doolforward(L,b):
  y=np.zeros(b.shape)
  bc=np.copy(b)
  n=len(b)
  y[0]=bc[0]
  for p in range(1,n):
    y[p]=bc[p]-np.sum(y[:p]*L[p,:p])
  return y

def doolbackward(U,y):
  x=np.zeros(y.shape)
  n=len(y)
  x[n-1]=y[n-1]/U[n-1,n-1]
  for q in range(n-2,-1,-1):
    x[q]=(y[q]-np.sum(x[q+1:]*U[q,q+1:]))/U[q,q]
  return x

def inverse(A):
  L,U=doolittle(A)
  n=len(np.diag(A))
  Ainv=np.zeros(A.shape)
  for i in range(n):
    b=np.zeros(n)
    b[i]=1
    y=doolforward(L,b)
    x=doolbackward(U,y)
    Ainv[:,i]=x
  return Ainv

In part(3) we assess the value of $A*A^{-1}$ using the inbuilt function np.linalg.det() which calculates determinant of matrices. If the inverse is found correctly than the determinant should be close to 1.

In part(4) condition number is calculated using inbuilt functions np.linalg.norm() which calculates the norm of a matrix.

In part(5) an inbuil function np.linalg.inv() is used to calculate inverse. Also, %%timeit functions are used to determine the speed of computations.



#Results

(1) A is generated:

In [5]:
A=generate_a()

(2) Ainv is inverse matrix of A

In [6]:
Ainv=inverse(A)

(3)I is the product of A and its inverse
We can see that I is pretty close to identity matrix and the error in the determinant is between $10^{-13}$ and $10^{-14}$ orders of magnitude

In [8]:
I=np.matmul(A,Ainv)
print(1-np.linalg.det(I))

4.39648317751562e-14


(4)normA is the norm of A
normAinv is the norm of inverse of A
cond is condition number
cons has a magnitude of $10^3$ 

In [9]:
normA=np.linalg.norm(A)
normAinv=np.linalg.norm(Ainv)
cond=normA*normAinv
print(cond)

1232.7905702783369


(5)LU_inverse is cumputed using my code
numpy_inverse is calculated using numpy library

numpy_inverse is 40 times faster than LU_inverse

In [10]:
%%timeit -r5 -n100
A=generate_a()
LU_inverse=inverse(A)

100 loops, best of 5: 42.1 ms per loop


In [11]:
%%timeit -r5 -n100
A=generate_a()
numpy_inverse=np.linalg.inv(A)

100 loops, best of 5: 1.41 ms per loop


#Conclusions

It was found that the implementation of LU decomposition in this code was successful in finding iverse of a matrix with an error of $10^{-13}-10^{-14}. Since matrix $A$ has a condition number of order $10^3$ and the default precision of float is $10^{-16}, we can see that the results of parts(3) and (4) are consistent with each other: $10{-16}*1000=10^{-13}$

In part(5) numpy implementation of inverse was 40 times faster. This is because numpy uses special library LAPACK which is based on a simpler language C, so it is more optimized for such computations.